In [2]:
from vtk import *
import itk

In [5]:
from ipywidgets import interact
import matplotlib.pyplot as plt
import numpy as np

im = itk.imread("./data/BRATS_HG0015_T1C.mha")
im2 = itk.GetArrayFromImage(im)

# cast image into F,3
im = itk.CastImageFilter[
    itk.Image[itk.SS,3], itk.Image[itk.F, 3]
].New(Input=im)
im.Update()

im = itk.RescaleIntensityImageFilter.New(
    Input=im,
    OutputMinimum=0.0,
    OutputMaximum=1.0
)
im.Update()

im = itk.ThresholdImageFilter.New(
    Input=im,
    Lower=0.4,
)
im.Update()

# im = itk.CastImageFilter[
#     itk.Image[itk.F,3], itk.Image[itk.UC, 3]
# ].New(Input=im)
# im.Update()

# im = itk.ConnectedComponentImageFilter.New(
#     Input=im
# )
# im.Update()

# Anisotropic smooth of the image
# im = itk.GradientAnisotropicDiffusionImageFilter.New(
#     Input=im, 
#     NumberOfIterations=10, 
#     ConductanceParameter=10, 
#     TimeStep=0.125,
# )
# im.Update()

# im = itk.WatershedImageFilter.New(
#     Input=im,
#     Level=.5,
#     Threshold=.005,
# )
# im.Update()

im = itk.GetArrayFromImage(im)
mask = im > 0.4

@interact
def show(i = (0, len(im)-1)):
    image = im[i,:,:]
    image2 = im2[i,:,:]
    if mask is not None:
        m = mask[i,:,:]
        image = np.stack((image,) * 3, axis=-1)
        
        image[:,:,0][m] = image.max()
        image[:,:,1][m] = 0
        image[:,:,2][m] = 0
    plt.figure()
    plt.imshow(image)
    plt.figure()
    plt.imshow(image2)

interactive(children=(IntSlider(value=87, description='i', max=175), Output()), _dom_classes=('widget-interact…

In [2]:
def _check_valid_arg(val, name, available):
    if val not in available:
        raise f"{name}='{val}' is not a valid arg. Valid values are: {available}"
        
def load_volume(reader, min_intensity=0, intensity_coeff=0.5, color=(1.,1.,1.), opacity_window=4096, render_with="gl", interpolation="linear"):
    _check_valid_arg(render_with, "render_with", {'gl', 'gpu', 'cpu'})
    _check_valid_arg(interpolation, "interpolation", {'linear', 'nearest'})
    
    reader.Update()

    if render_with == "gl":
        mapper = vtkOpenGLGPUVolumeRayCastMapper()
    elif render_with == 'gpu':
        mapper = vtkGPUVolumeRayCastMapper() 
    elif render_with == 'cpu':
        mapper = vtkFixedPointVolumeRayCastMapper()
    else:
        raise "unexpected"
        
    mapper.SetInputConnection(reader.GetOutputPort())
    mapper.SetSampleDistance(0.5)
    mapper.SetBlendModeToMaximumIntensity()

    data_min, data_max = reader.GetOutput().GetScalarRange()
    
    seg_min, seg_max = min_intensity, min_intensity + (1-intensity_coeff) * (data_max-min_intensity)
    fct_color = vtkColorTransferFunction()
    fct_color.AddRGBSegment(seg_min, *(0., 0., 0.),
                            seg_max, *color)
    
    fct_opacity = vtkPiecewiseFunction()
    fct_opacity.AddSegment(seg_min, 0.,
                           seg_max, 1.)
    
    props = vtkVolumeProperty()
    props.SetIndependentComponents(True)
    props.SetColor(fct_color)
    props.SetScalarOpacity(fct_opacity)

    if interpolation == "linear":
        props.SetInterpolationTypeToLinear()
    elif interpolation == 'nearest':
        props.SetInterpolationTypeToNearest()
    else:
        raise "unexpected"

    volume = vtkVolume()
    volume.SetMapper(mapper)
    volume.SetProperty(props)
    
    return volume, props

In [3]:
reader = vtkMetaImageReader()
reader.SetFileName("./data/BRATS_HG0015_T1C.mha")

In [4]:
volume, _ = load_volume(reader, intensity_coeff=0.4)
volume_red, _ = load_volume(reader, intensity_coeff=0.8, min_intensity=1000, color=[1,0,0])

ren = vtkRenderer()
ren.AddVolume(volume)
ren.AddVolume(volume_red)

renWin = vtkRenderWindow()
renWin.AddRenderer(ren)

iren = vtkRenderWindowInteractor()
iren.SetRenderWindow(renWin)
iren.Initialize()

# iren.AddObserver('TimerEvent', cb.execute)
# timerId = iren.CreateRepeatingTimer(100);

renWin.Render()

iren.Start()